In [23]:
from bs4 import BeautifulSoup
import requests
import math
import pandas as pd
import numpy as np
from urllib.parse import urlparse
import sqlite3
key_fd = open('./과제data/kakaomapkey.txt', mode='r')
kmap_key = key_fd.read(100)
key_fd.close()

In [24]:
key_fd = open('./과제data/seoulapikey.txt', mode='r')
seoul_key = key_fd.read(100)
key_fd.close()

In [120]:
seoul_url = 'http://openapi.seoul.go.kr:8088/'
url = f'{seoul_url}{seoul_key}/xml/Corona19Status/1/1000'

In [121]:
url

'http://openapi.seoul.go.kr:8088/694f57486f686a6b33387443706a41/xml/Corona19Status/1/1000'

In [26]:
result = requests.get(url)
soup = BeautifulSoup(result.text, 'xml')
soup.find('MESSAGE').get_text()

'정상 처리되었습니다'

In [27]:
total_count = int(soup.find('list_total_count').get_text())

In [28]:
total_count/1000

18.253

In [29]:
count = math.ceil(total_count/1000)
count

19

In [30]:
rows = soup.find_all('row')

In [31]:
rows[0].find('CORONA19_DATE').text

'12.28.'

In [32]:
rows[0].find('CORONA19_AREA').text

'송파구'

In [33]:
rows[0].find('CORONA19_ID').text

'18253'

In [34]:
rows[0].find('CORONA19_CONTACT_HISTORY').text

'동부구치소 관련'

In [35]:
id_list, area_list, date_list,histroy_list = [],[],[],[]
a_url = f'{seoul_url}{seoul_key}/xml/Corona19Status'
x = 1
for i in range(count):  
    url = f'{a_url}/{x}/{x+999}'
    x += 1000
    result = requests.get(url)
    soup = BeautifulSoup(result.text, 'xml')
    rows = soup.find_all('row')
    for row in rows:
        id_list.append(row.find('CORONA19_ID').text)
        area_list.append(row.find('CORONA19_AREA').text)
        date_list.append(row.find('CORONA19_DATE').text)
        histroy_list.append(row.find('CORONA19_CONTACT_HISTORY').text)

In [36]:
df = pd.DataFrame({
    '연번' : id_list,
    '확진일' : date_list,
    '지역' : area_list,
    '접촉력' : histroy_list
})

In [37]:
len(histroy_list)

18253

In [38]:
df['지역'] = df['지역'].str.replace(' ','')

In [39]:
df['지역'][df['지역'] == '중로구']

2630    중로구
Name: 지역, dtype: object

In [40]:
df['지역'][df['지역'] == '중로구'] = '종로구'

In [41]:
df['확진일'] = '2020.' + df['확진일']

In [42]:
df['확진일'] = df['확진일'].str.replace('.','-').str[:-1]

In [43]:
df['확진일'] = pd.to_datetime(df['확진일'])

In [44]:
df['확진일'] = df['확진일'].astype(str)

In [47]:
df = df.sort_values(by='확진일')

In [48]:
df.reset_index(drop=True,inplace=True)

In [50]:
df.tail()

,연번,확진일,지역,접촉력
18248,17899,2020-12-28,관악구,기타 확진자 접촉
18249,17898,2020-12-28,용산구,기타 확진자 접촉
18250,17897,2020-12-28,도봉구,기타 확진자 접촉
18251,17895,2020-12-28,양천구,기타 확진자 접촉
18252,18253,2020-12-28,송파구,동부구치소 관련


In [51]:
conn = sqlite3.connect('./DB/covid-19.db')
cur = conn.cursor()

In [52]:
sql_table = '''
    create table '서울시 확진자 현황'(
        '연번' text not null primary key, '확진일' text not null,
        '지역' text not null, 접촉력 text not null
    )
'''

In [53]:
cur.execute(sql_table)
conn.commit()

In [54]:
sql_insert = 'insert into "서울시 확진자 현황" values(?,?,?,?)'

In [55]:
for i in df.index:
    params = list(df.loc[i])
    cur.execute(sql_insert, params)
    conn.commit()

In [56]:
conn.close()

In [57]:
conn = sqlite3.connect('./DB/covid-19.db')
cur = conn.cursor()
cur.execute(f'select * from "서울시 확진자 현황"')
rows = cur.fetchall()
conn.close()

In [58]:
df = pd.DataFrame(rows, columns=['연번','확진일','지역','접촉력'])

In [59]:
df

,연번,확진일,지역,접촉력
0,1,2020-01-24,강서구,해외 접촉
1,4,2020-01-30,마포구,해외 접촉
2,2,2020-01-30,중랑구,해외 접촉
3,3,2020-01-30,종로구,#3 접촉
4,5,2020-01-31,성북구,#5 접촉
...,...,...,...,...
18248,17899,2020-12-28,관악구,기타 확진자 접촉
18249,17898,2020-12-28,용산구,기타 확진자 접촉
18250,17897,2020-12-28,도봉구,기타 확진자 접촉
18251,17895,2020-12-28,양천구,기타 확진자 접촉


In [60]:
sam = df[df['지역'] == '강서구'].pivot_table('연번','접촉력',aggfunc="count")

In [61]:
sam.sort_values(by='연번', ascending=False).head(10)

,연번
접촉력,
기타 확진자 접촉,330
감염경로 조사중,215
강서구 교회 관련,163
강서구 댄스교습관련 시설,153
타시도 확진자 접촉,45
확인 중,26
8.15서울도심집회,18
강서구 소재 병원관련(11월),16
강서구 소재 댄스교습시설 연관 병원,16


In [62]:
list(sam.sort_values(by='연번', ascending=False).head(10).index)

['기타 확진자 접촉',
 '감염경로 조사중',
 '강서구 교회 관련',
 '강서구 댄스교습관련 시설',
 '타시도 확진자 접촉',
 '확인 중',
 '8.15서울도심집회',
 '강서구 소재 병원관련(11월)',
 '강서구 소재 댄스교습시설 연관 병원',
 '관악구 소재 음식점 관련']

In [63]:
sam.sort_values(by='연번', ascending=False).head(10)['연번']

접촉력
기타 확진자 접촉              330
감염경로 조사중               215
강서구 교회 관련              163
강서구 댄스교습관련 시설          153
타시도 확진자 접촉              45
확인 중                    26
8.15서울도심집회              18
강서구 소재 병원관련(11월)        16
강서구 소재 댄스교습시설 연관 병원     16
관악구 소재 음식점 관련           14
Name: 연번, dtype: int64

In [64]:
a_d = df.pivot_table('연번', '지역', aggfunc='count')

In [10]:
a_d.rename({'연번':'확진자수'},axis=1, inplace=True)

In [11]:
city_list = []
for i in a_d.index:
    if i.find('구') != -1 :
        city_list.append(i+'청')

In [12]:
addr_list = []
for i in city_list:
    url = 'https://dapi.kakao.com/v2/local/search/keyword.json?query=' + i
    result = requests.get(urlparse(url).geturl(),headers={"Authorization": "KakaoAK "+kmap_key}).json()
    addr = result['documents'][0]['address_name']
    addr_list.append(addr)

In [84]:
lat_list =[] ; lng_list = [] 
for i in addr_list:
    url = 'https://dapi.kakao.com/v2/local/search/address.json?query=' + i
    result = requests.get(urlparse(url).geturl(),headers={"Authorization": "KakaoAK "+kmap_key}).json()
    lat = float(result['documents'][0]['address']['y'])
    lng = float(result['documents'][0]['address']['x'])
    lat_list.append(lat)
    lng_list.append(lng)

In [85]:
gu_df = pd.DataFrame({
    '지역' : city_list,
    '주소' : addr_list,
    'lat' : lat_list,
    'lng' : lng_list
})

In [86]:
gu_df.to_csv('static/data/서울구청리스트.csv')

In [87]:
a_d

,확진자수
지역,
강남구,910
강동구,549
강북구,409
강서구,1307
관악구,963
광진구,434
구로구,592
금천구,293
기타,851


In [67]:
d_day = df[df['지역'] == '강서구'].pivot_table('연번','확진일',aggfunc="count")

In [68]:
ten_day = d_day.sort_values(by='확진일', ascending=False).head(10)

In [95]:
list(ten_day.index)[::-1]


['2020-12-19',
 '2020-12-20',
 '2020-12-21',
 '2020-12-22',
 '2020-12-23',
 '2020-12-24',
 '2020-12-25',
 '2020-12-26',
 '2020-12-27',
 '2020-12-28']

'2020-12-28'

In [ ]:
ten_day = d_day.sort_values(by='확진일', ascending=False).head(10)
ten_date = list(ten_day.index)
ten_val = list(ten_day['연번'].values)